In [ ]:
from cobraCoach import CobraCoach
import pathlib
import numpy as np
import logging
import engineer as eng

In [ ]:
mod = 'SP01'
cc = CobraCoach('fpga', loadModel=False)
cc.loadModel(version=mod, moduleVersion=None)
eng.setCobraCoach(cc)

In [ ]:
eng.setConstantOntimeMode()

In [ ]:
eng.setConstantSpeedMode(maxSegments=50, maxSteps=100)

In [ ]:
mmTheta = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/SP01_mmThetaFast4.npy')
mmPhi = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/SP01_mmPhiFast2.npy')
mmThetaSlow = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/SP01_mmThetaSlow2.npy')
mmPhiSlow = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/SP01_mmPhiSlow2.npy')
eng.setConstantSpeedMaps(mmTheta, mmPhi, mmThetaSlow, mmPhiSlow)

In [ ]:
cc.visibleIdx

In [ ]:
# update center positions from home, not accurately
eng.convertXML(f'{mod}.xml')

In [ ]:
cc.showStatus()

In [ ]:
cc.setScaling(False)

In [ ]:
# enable motor scaling and set parameters
cc.setScaling(True, thetaScaleFactor=4.0, phiScaleFactor=4.0, minThetaSteps=1, minPhiSteps=1)

In [ ]:
cc.setScaling(True, thetaScaleFactor=4.0, phiScaleFactor=4.0, minThetaSteps=20, minPhiSteps=20)

In [ ]:
cc.setScaling(True, thetaScaleFactor=2.0, phiScaleFactor=2.0, minThetaSteps=200, minPhiSteps=200)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling()

In [ ]:
# switch to phi mode
eng.setPhiMode()

In [ ]:
# generate a quick motormap and set geometry
eng.makePhiMotorMaps(f'{mod}_phi.xml', steps=50, totalSteps=6000, repeat=1, fast=False)

In [ ]:
# set phi geometry from previous run
cc.setPhiGeometryFromRun('/data/MCS/20201029_013', onlyIfClear=False)

In [ ]:
# move phi to home position
cc.moveToHome(cc.goodCobras, phiEnable=True)

In [ ]:
# phi on time search
eng.phiOnTimeSearch(f'{mod}_phi_search.xml', iteration=3, repeat=1)

In [ ]:
# generate new motormaps
#eng.makePhiMotorMaps(f'{mod}_phi800.xml', steps=800, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi500.xml', steps=500, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi200.xml', steps=200, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps(f'{mod}_phi100.xml', steps=100, totalSteps=8000, repeat=1)
#eng.makePhiMotorMaps(f'{mod}_phi50.xml', steps=50, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi20.xml', steps=20, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi10.xml', steps=10, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi5.xml', steps=5, totalSteps=6000, repeat=3)

In [ ]:
eng.makePhiMotorMaps(f'{mod}_phi_fast.xml', steps=50, totalSteps=6000, repeat=1, fast=True)

In [ ]:
eng.makePhiMotorMaps(f'{mod}_phi_test.xml', steps=25, totalSteps=6000, repeat=1, fast=True, phiOnTime=phiOt)

In [ ]:
eng.makePhiMotorMaps(f'{mod}_phi_test.xml', steps=50, totalSteps=6000, repeat=1, fast=False, phiOnTime=phiOt)

In [ ]:
# move phi by angles
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(165), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# move by a angle
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(90), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# convergence test
moves = eng.phiConvergenceTest(cc.goodIdx, margin=15.0, runs=51, tries=8, fast=False, tolerance=0.1)

In [ ]:
# move phi to 60 degree for theta test
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.pi/3, relative=False, local=False, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# phi on-time scan
dataPath, ontimes, angles, speeds = eng.phiOntimeScan(speed=np.deg2rad(0.12), steps=10, totalSteps=6000, repeat=1, scaling=3.0)

In [ ]:
dataPath, ontimes, angles, speeds = eng.phiOntimeScan(speed=np.deg2rad(0.06), steps=20, totalSteps=9000, repeat=1, scaling=4.0, tolerance=np.deg2rad(3.0))

In [ ]:
# swtich to theta mode
eng.setThetaMode()

In [ ]:
# generate a quick motormap and set geometry
eng.makeThetaMotorMaps(f'{mod}_theta.xml', steps=200, totalSteps=10000, repeat=1, fast=False)

In [ ]:
# set theta geometry from previous run
cc.setThetaGeometryFromRun('/data/MCS/20201030_025', onlyIfClear=False)

In [ ]:
# move theta to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True, thetaCCW=False)

In [ ]:
# set current angle to zeros, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=0)

In [ ]:
# theta on time search
eng.thetaOnTimeSearch(f'{mod}_theta_search.xml', iteration=3, repeat=1)

In [ ]:
# generate new motormaps
#eng.makeThetaMotorMaps(f'{mod}_theta800.xml', steps=800, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta500.xml', steps=500, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta200.xml', steps=200, totalSteps=12000, repeat=1)
#eng.makeThetaMotorMaps(f'{mod}_theta100.xml', steps=100, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta50.xml', steps=50, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps(f'{mod}_theta.xml', steps=50, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps(f'{mod}_theta.xml', steps=25, totalSteps=12000, repeat=3, fast=True)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta5.xml', steps=5, totalSteps=12000, repeat=3, force=True)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta_test.xml', steps=100, totalSteps=12000, repeat=3, fast=True, thetaOnTime=thetaOt)

In [ ]:
for ot in [0.06, 0.08, 0.1, 0.12, 0.14]:
    eng.makeThetaMotorMaps(f'{mod}_theta_test.xml', steps=25, totalSteps=5000, repeat=1, fast=True, thetaOnTime=ot)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling(cc.allCobras)

In [ ]:
# move by angles
dataPath, diffAngles, moves = eng.moveThetaAngles(cc.goodIdx, np.pi/3, tries=6)

In [ ]:
# convergence test
moves = eng.thetaConvergenceTest(cc.goodIdx, margin=15.0, runs=5, tries=10, fast=False, tolerance=0.1)

In [ ]:
# move theta arms outward in theta mode
#eng.maxSteps = 100
tht = np.empty(cc.nCobras)
tht[0::2] = np.pi*3/2 - np.pi/3
tht[1::2] = np.pi/2 - np.pi/3
#eng.moveThetaAngles(cc.goodIdx, tht[cc.goodIdx], local=False)
eng.moveThetaAngles2(cc.goodIdx, tht[cc.goodIdx], local=False, tolerance=0.002, tries=10, fast=False, newDir=True)

In [ ]:
# theta on-time scan
dataPath, ontimes, angles, speeds = eng.thetaOntimeScan(speed=np.deg2rad(0.06), steps=20, totalSteps=15000, repeat=1, scaling=3.0, tolerance=np.deg2rad(3.0))

In [ ]:
# theta on-time scan
dataPath, ontimes, angles, speeds = eng.thetaOntimeScan(speed=np.deg2rad(0.12), steps=10, totalSteps=10000, repeat=1, scaling=3.0)

In [ ]:
# update cobra geometry
eng.updateXML(f'{mod}_new.xml', '/data/MCS/20201029_004', '/data/MCS/20201029_001')

In [ ]:
# swtich to normal mode
eng.setNormalMode()

In [ ]:
# move to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True, phiEnable=True, thetaCCW=False)
#cc.moveToHome([cc.allCobras[46]], thetaEnable=True, phiEnable=True, thetaCCW=True)

In [ ]:
cc.getCurrentPositions()

In [ ]:
# set current angle, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=0)

In [ ]:
# move by angles
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, np.pi/2, np.pi/3, relative=False, local=True, tolerance=0.1, tries=6, homed=False, newDir=True)

In [ ]:
# move by positions, go home first and do safe movement
tries = 12
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=tries, thetaMarginCCW=0.1, homed=True, newDir=True)

In [ ]:
# move by positions again
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=8, thetaMarginCCW=0.1, homed=False, newDir=True)

In [ ]:
# 2D target convergence test
targets, moves = eng.convergenceTest(cc.goodIdx, runs=3, thetaMargin=np.deg2rad(15.0), phiMargin=np.deg2rad(15.0), tries=8, tolerance=0.2)

In [ ]:
targets, moves = eng.convergenceTestY(cc.goodIdx, runs=3, thetaMargin=np.deg2rad(15.0), phiMargin=np.deg2rad(15.0), tries=8, tolerance=0.2, threshold=3.0)

In [ ]:
# save results in npy format
np.save('targets2', targets)
np.save('moves2', moves)

In [ ]:
# save results for convergence test in csv format
runs, nCobras, tries = moves.shape
mTargets = np.zeros(runs*nCobras, dtype=np.dtype(dict(names=['runs', 'cobra', 'x', 'y', 'theta', 'phi'], formats=['i4', 'i4', 'f4', 'f4', 'f4', 'f4'])))
mMoves = np.zeros(runs*nCobras*tries, dtype=np.dtype(dict(names=['runs', 'cobra', 'tries', 'x', 'y', 'theta', 'thetaSteps', 'thetaOntime', 'thetaFast', 'phi', 'phiSteps', 'phiOntime', 'phiFast'], formats=['i4', 'i4', 'i4', 'f4', 'f4', 'f4', 'i4', 'f4', 'i4', 'f4', 'i4', 'f4', 'i4'])))

for m in range(runs):
    mPositions = cc.pfi.anglesToPositions(cc.goodCobras, targets[m,:,0], targets[m,:,1])
    for n in range(nCobras):
        mTargets[m*nCobras+n] = m+1, cc.goodIdx[n]+1, mPositions[n].real, mPositions[n].imag, np.rad2deg(targets[m,n,0]), np.rad2deg(targets[m,n,1])
        for p in range(tries):
            mMoves[(m*nCobras+n)*tries+p] = m+1, cc.goodIdx[n]+1, p+1, moves[m,n,p]['position'].real, moves[m,n,p]['position'].imag, np.rad2deg(moves[m,n,p]['thetaAngle']), moves[m,n,p]['thetaSteps'], moves[m,n,p]['thetaOntime'], moves[m,n,p]['thetaFast'], np.rad2deg(moves[m,n,p]['phiAngle']), moves[m,n,p]['phiSteps'], moves[m,n,p]['phiOntime'], moves[m,n,p]['phiFast']

np.savetxt(f'targets2.csv', mTargets, fmt='%d %d %.2f %.2f %.2f %.2f', header='runs cobra x y theta phi')
np.savetxt(f'moves2.csv', mMoves, fmt='%d %d %d %.2f %.2f %.2f %d %.4f %d %.2f %d %.4f %d', header='runs cobra tries x y theta thetaSteps thetaOntime thetaFast phi phiSteps phiOntime phiFast')

In [ ]:
# goto safe position
tht = np.empty(cc.nCobras)
tht[0::2] = np.pi*3/2
tht[1::2] = np.pi/2
#dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, newDir=True)
#dataPath, thetas, phis, moves = eng.moveThetaPhi2(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, thetaThreshold=0.2, phiThreshold=0.2, newDir=True)
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.2, tries=10, homed=False, newDir=True)

In [ ]:
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.2, tries=6, homed=False, newDir=False)

In [ ]:
eng.moveDeltaAngles(cc.goodIdx, thetaAngles=0.1, phiAngles=0.1, thetaThreshold=0.1, phiThreshold=0.1)

In [ ]:
cc.cobraInfo[29], cc.cobraInfo[15], cc.thetaInfo[54]

In [ ]:
cobras = [cc.allCobras[47]]
#cc.moveToHome(cobras, thetaEnable=True, phiEnable=True, thetaCCW=False)
#cc.moveDeltaAngles(cobras, thetaAngles=None, phiAngles=np.deg2rad(30), thetaFast=True, phiFast=True)
cc.moveDeltaAngles(cobras, thetaAngles=np.deg2rad(60), phiAngles=None, thetaFast=True, phiFast=True)
#dataPath, thetas, phis, moves = eng.moveThetaPhi2(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, thetaThreshold=0.2, phiThreshold=0.2, newDir=True)

In [ ]:
cc.pfi.moveThetaPhi(cobras, [np.deg2rad(60)], [0], [0.3740908], [0.5304951], True, True, doRun=False)

In [ ]:
cc.showStatus()

In [ ]:
targets[0,0], moves[0,0], targets[0,35], moves[0,35], targets[0,37], moves[0,37], targets[0,53], moves[0,53]

In [ ]:
cc.moveInfo

In [ ]:
cc.pfi.calibModel.cobraStatus(31)

In [ ]:
cc.pfi.calibModel.setCobraStatus(31, brokenTheta=True)
cc.pfi.calibModel.createCalibrationFile('SC32_5.xml')

In [ ]:
np.rad2deg((cc.calibModel.tht1 - cc.calibModel.tht0 + np.pi) % (np.pi*2) - np.pi)

In [ ]:
cc.goodIdx

In [ ]:
moves[30,10:20,:]

In [ ]:
cc.moveSteps(cc.goodCobras, 0, -5000)

In [ ]:
cc.moveInfo, cc.phiInfo

In [ ]:
np.abs(cc.cobraInfo['position'] - cc.calibModel.centers)

In [ ]:
cc.cobraInfo

In [ ]:
cc.pfi.positionsToAngles(cc.allCobras, cc.cobraInfo['position'])[2][2,0] & 17

In [ ]:
cc.calibModel.L1, cc.calibModel.L2

In [ ]:
np.diff(np.rad2deg(angles[0,0])) / 250

In [ ]:
np.rad2deg(angles[0,0]), ontimes[0,0], np.rad2deg(speeds[0,0])

In [ ]:
np.rad2deg(speeds[0,0,:])

In [ ]:
ontimes[1,0,:]

In [ ]:
thetaOt = np.array([[0.0798, 0.0687, 0.0904, 0.0404, 0.0567, 0.0471, 0.0626, 0.0589,
        0.064 , 0.0645, 0.0464, 0.0447, 0.0563, 0.0372, 0.0441, 0.0613,
        0.0739, 0.0492, 0.1072, 0.052 , 0.0579, 0.0461, 0.0988, 0.0698,
        0.0546, 0.0648, 0.055 , 0.0458, 0.0519, 0.0563, 0.055 , 0.0492,
        0.0659, 0.0614, 0.0762, 0.0452, 0.0433, 0.0562, 0.0485, 0.0521,
        0.0502, 0.068 , 0.0561, 0.0669, 0.048 , 0.0575, 0.0658, 0.0436,
        0.0694, 0.0454, 0.0515, 0.0709, 0.0502, 0.0587, 0.0477, 0.1135,
        0.0494],
       [0.0808, 0.0551, 0.0812, 0.0442, 0.0597, 0.0537, 0.0623, 0.0588,
        0.0764, 0.0624, 0.0521, 0.0459, 0.062 , 0.0391, 0.0477, 0.0685,
        0.0714, 0.0505, 0.0956, 0.0412, 0.0574, 0.0521, 0.0926, 0.0766,
        0.0524, 0.0898, 0.0478, 0.0514, 0.0519, 0.0658, 0.0476, 0.0685,
        0.069 , 0.0807, 0.0702, 0.0489, 0.0406, 0.0579, 0.0554, 0.0511,
        0.0535, 0.0841, 0.056 , 0.0629, 0.0516, 0.0501, 0.05  , 0.0539,
        0.0667, 0.0459, 0.0564, 0.0907, 0.0559, 0.07  , 0.0496, 0.0758,
        0.0454]])

In [ ]:
phiOt = np.array([[0.0366, 0.0198, 0.0261, 0.019 , 0.0535, 0.0196, 0.0232, 0.0249,
        0.019 , 0.0289, 0.0298, 0.023 , 0.0307, 0.0257, 0.0246, 0.0221,
        0.0404, 0.0514, 0.0309, 0.0155, 0.0204, 0.017 , 0.0284, 0.0303,
        0.0247, 0.032 , 0.0211, 0.0235, 0.023 , 0.0331, 0.0258, 0.017 ,
        0.0319, 0.02  , 0.0391, 0.0145, 0.0248, 0.0185, 0.0342, 0.0258,
        0.0184, 0.0273, 0.0277, 0.0244, 0.0237, 0.0181, 0.0345, 0.0238,
        0.0255, 0.0254, 0.0234, 0.0163, 0.0284, 0.0253, 0.0266, 0.0245,
        0.0191],
       [0.0317, 0.0207, 0.0255, 0.0211, 0.0294, 0.0213, 0.0333, 0.024 ,
        0.017 , 0.0297, 0.0202, 0.0209, 0.03  , 0.02  , 0.028 , 0.017 ,
        0.0405, 0.0333, 0.0255, 0.015 , 0.0205, 0.0212, 0.036 , 0.0566,
        0.0305, 0.03  , 0.0244, 0.0289, 0.0203, 0.0281, 0.0265, 0.0252,
        0.0323, 0.0304, 0.0362, 0.0185, 0.029 , 0.0191, 0.0336, 0.0275,
        0.0204, 0.0235, 0.0269, 0.0225, 0.0267, 0.0218, 0.0288, 0.0249,
        0.024 , 0.0217, 0.0221, 0.016 , 0.028 , 0.034 , 0.0325, 0.0265,
        0.0365]])